In [1]:
import execute_tensor as execute
from execute_tensor import execute as exe

2024-01-03 11:26:47.881071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 11:26:48.057445: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 11:26:48.859612: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.3/lib64
2024-01-03 11:26:48.859711: W te

In [2]:
from execute import tox_process

In [4]:
from Module import custom_layers
import tensorflow as tf

BERT_tensor = custom_layers.BERT_tensor_small

bert_layer = BERT_tensor(256,8,1024,strat_index=7)

In [4]:
A = tox_process('temp')

In [5]:
import pickle
from execute_tensor import predict

with open('./BERT/SMILE/1M_random_ZINC_word2index.pkl','rb') as file:
    word2idx = pickle.load(file)

In [6]:
from tensorflow.keras import Model
inputs = tf.keras.layers.Input(shape = (200,),dtype=tf.int32)
outputs = bert_layer(inputs,None)

model = Model(inputs = [inputs], outputs = [outputs])

In [7]:
with open('./BERT/SMILE/small_Pre_BERT.pkl','rb') as file:
    paras = pickle.load(file)

In [8]:
model.set_weights(paras)
paras = model.get_weights()
bert_layer.set_weights(paras)

In [9]:
import numpy as np
def Task_mask(num_task):
    result = np.zeros([200,200])
    for i in range(num_task):
        for j in range(200):
            if j == i:
                continue
            else:
                result[j][i] = 1
    return result

In [10]:
mask = Task_mask(7)

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.metrics import AUC
Models = []

Input = tf.keras.layers.Input(200,)
hidden = bert_layer(Input,att_mask = mask)
output = hidden[:,0]
output = tf.keras.layers.Dropout(0.3)(output)
output = tf.keras.layers.Dense(100,activation = 'gelu')(output)
output = tf.keras.layers.Dense(1,activation = 'sigmoid')(output)
globals()[f'Task{0}_model'] = Model(inputs = [Input],outputs = [output])
globals()[f'Task{0}_model'].compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),loss = 'binary_crossentropy',metrics=['acc',AUC(name='auc')])
Models.append(globals()[f'Task{0}_model'])

for i in range(9):
    i = i+1
    Input = tf.keras.layers.Input(200,)
    hidden = bert_layer(Input,att_mask = mask)
    output = hidden[:,i]
    output = tf.keras.layers.Dropout(0.3)(output)
    output = tf.keras.layers.Dense(100,activation = 'gelu')(output)
    output = tf.keras.layers.Dense(1,activation = 'sigmoid')(output)
    globals()[f'Task{i}_model'] = Model(inputs = [Input],outputs = [output])
    globals()[f'Task{i}_model'].compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),loss = 'binary_crossentropy',metrics=['acc',AUC(name='auc')])
    Models.append(globals()[f'Task{i}_model'])
    

In [13]:
import pickle
with open('./BERT/SMILE/1M_random_ZINC_word2index.pkl','rb') as file:
    word2idx = pickle.load(file)
def w2idx(X_train,task_num = 2):
    res = []
    for i in X_train:
        temp = []
        for k in range(task_num):
            temp.append(k+len(word2idx)+1)
        temp.append(1)
        for j in i:
            temp.append(word2idx[j])
        res.append(temp)
    res = tf.keras.utils.pad_sequences(res,padding='post',maxlen=200)
    return res
        

In [12]:
import random
Tox_names = ['AMES','ClinTox','hERG','DILI','Skin_Reaction','hERG_Karim']
x_trains = [] 
y_trains = []
x_vals = []
y_vals = []
len_20s = []
task_labels = []
val_task = []
for index1,i in enumerate(Tox_names):
    try:
        
        A = tox_process(i)
        x_train,x_val,y_train,y_val,len_20 = A.AIS_process(number_of_task=10,token='AIS')
        index = 0
        res = []
        for i in len_20:
            res.append(np.average(y_val[index:index+i]))
            index = index+i
        y_val = np.array(res)
        
        temp_zip = list(zip(x_train,y_train))
        random.shuffle(temp_zip)
        x_train,y_train = zip(*temp_zip)
        
        
        temp_x_trains = x_train[:int(len(x_train)/32)*32]
        temp_x_trains = np.reshape(temp_x_trains,[-1,32,200])
        x_trains += list(temp_x_trains) + list(x_train[int(len(x_train)/32)*32:])
        
        temp_y_trains = y_train[:int(len(y_train)/32)*32]
        temp_y_trains = np.reshape(temp_y_trains,[-1,32])
        tmp = list(temp_y_trains) + list(y_train[int(len(y_train)/32)*32:])
        y_trains += tmp
        
        
        
        task_labels+=[index1]*len(tmp)
        
        
        y_vals.append(y_val)
        x_vals.append(x_val)
        len_20s.append(len_20)
        val_task.append(index1)
    except:
        pass

In [14]:
import random
Tox_names = ['AMES','ClinTox','hERG','DILI','hERG_Karim','Skin Reaction','Carcinogens_Lagunin']
x_trains = [] 
y_trains = []
x_vals = []
y_vals = []
len_20s = []
task_labels = []
val_task = []
for index1,i in enumerate(Tox_names):
    with open(f'./Tox_data/SMILE_Tox_data/{i}','rb') as file:
        corpus = pickle.load(file)
    spliter = tox_process('temp',0.2,204)
    x_train,x_val,y_train,y_val,len_20 = spliter.train_val_split(corpus[0],word2idx,7)
    print(y_train)
    """
    with open(f'./data_sets/{i}_train','rb') as file:
        X_train,y_train,_ = pickle.load(file)
        x_train = w2idx(X_train,0)
    with open(f'./data_sets/{i}_val','rb') as file:
        X_val,y_val,len_20 = pickle.load(file)
        x_val = w2idx(X_val,0)"""
    index = 0
    res = []
    for i in len_20:
        res.append(np.average(y_val[index:index+i]))
        index = index+i
    y_val = np.array(res)
    temp_zip = list(zip(x_train,y_train))
    random.shuffle(temp_zip)
    x_train,y_train = zip(*temp_zip)
    
    
    temp_x_trains = x_train[:int(len(x_train)/32)*32]
    temp_x_trains = np.reshape(temp_x_trains,[-1,32,200])
    x_trains += list(temp_x_trains) + list(x_train[int(len(x_train)/32)*32:])
    
    temp_y_trains = y_train[:int(len(y_train)/32)*32]
    temp_y_trains = np.reshape(temp_y_trains,[-1,32])
    tmp = list(temp_y_trains) + list(y_train[int(len(y_train)/32)*32:])
    y_trains += tmp
    
    
    
    task_labels+=[index1]*len(tmp)
    
    
    y_vals.append(y_val)
    x_vals.append(x_val)
    len_20s.append(len_20)
    val_task.append(index1)

[1 1 1 ... 1 1 1]
[0 0 0 ... 0 0 0]
Unexpected :  [Cu+2]
Unexpected :  [Mn+2]
Unexpected :  [H-]
Unexpected :  [SbH]
[0. 0. 0. ... 1. 1. 1.]
[0. 0. 0. ... 1. 1. 1.]
[0 0 0 ... 0 0 0]
[1 1 1 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [99]:
len(y_vals[0])

1456

In [100]:
y_trains[0]

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1])

In [15]:
import random

temp_zip = list(zip(x_trains,y_trains,task_labels))
random.shuffle(temp_zip)

x_trains,y_trains,task_labels = zip(*temp_zip)

In [16]:
word2idx

{'<start>': 1,
 '<end>': 2,
 '<unknown1>': 3,
 '<unknown2>': 4,
 '<unknown3>': 5,
 '<unknow4>': 6,
 '<unknown5>': 7,
 'C': 8,
 'n': 9,
 '1': 10,
 'c': 11,
 '(': 12,
 'N': 13,
 '2': 14,
 '[C@H]': 15,
 '[nH]': 16,
 'S': 17,
 ')': 18,
 '=': 19,
 'O': 20,
 '[C@@H]': 21,
 'F': 22,
 '[O-]': 23,
 '[N@@H+]': 24,
 '3': 25,
 '[N@H+]': 26,
 'o': 27,
 '4': 28,
 'Br': 29,
 '[nH+]': 30,
 '[NH+]': 31,
 '5': 32,
 '[C@]': 33,
 '[C@@]': 34,
 's': 35,
 '-': 36,
 '[NH3+]': 37,
 '/': 38,
 '\\': 39,
 'Cl': 40,
 '[N+]': 41,
 '#': 42,
 '[N-]': 43,
 '[NH2+]': 44,
 '[P@]': 45,
 '[P@@]': 46,
 '[n-]': 47,
 '[n+]': 48,
 '[N@+]': 49,
 '[N@@+]': 50,
 'I': 51,
 'P': 52,
 '[PH+]': 53,
 '6': 54,
 '[S@@]': 55,
 '[Si]': 56,
 '[S@]': 57,
 '[o+]': 58,
 '[S-]': 59,
 '[O+]': 60,
 '7': 61,
 '[S@@+]': 62,
 '[S@+]': 63,
 '[S+]': 64,
 '[P@H]': 65,
 '[P@@H]': 66,
 '[SH]': 67,
 '[s+]': 68,
 '[P+]': 69,
 '[PH]': 70,
 '8': 71,
 '[Hg]': 72,
 '.': 73,
 '[Na]': 74,
 '[Cl+3]': 75,
 '[cH-]': 76,
 '[CH-]': 77,
 '[CH2-]': 78,
 '[C-]': 79,


In [17]:
x_trains[0][0]

array([156, 157, 158, 159, 160, 161, 162,   1,  13,   8,  12,  19,  20,
        18,  13,  13,   8,  12,  13,  18,  19,  20,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [18]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 200)]             0         
                                                                 
 bert_tensor_small_1 (BERT_t  (None, 200, 256)         4519232   
 ensor_small)                                                    
                                                                 
Total params: 4,519,232
Trainable params: 4,519,232
Non-trainable params: 0
_________________________________________________________________


In [19]:
x_trains[1][0]

array([160, 161, 162, 163, 164, 165, 166,   1,  11,  10,  11,  12,  11,
        11,  11,  12,  29,  18,  11,  10,  18,   8,  12,   8,   8,  13,
        12,   8,  18,   8,  18,  11,  10,  11,  11,  11,  11,   9,  10,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [20]:
from tqdm import tqdm
import sys
batch_size = 32
epochs = 25


loss_fn = tf.keras.losses.BinaryCrossentropy()
loss_fn_reg = tf.keras.losses.MeanSquaredError()
acc = np.array([0])



for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    loss_list = [[] for i in range(len(Tox_names))]
    acc_list = [[] for i in range(len(Tox_names))]
    auc_list = [[] for i in range(len(Tox_names))]
    for i in range(0, len(temp_zip)):
        batch_images = x_trains[i]
        batch_images = tf.reshape(batch_images,[-1,200])
        batch_labels = y_trains[i]
        batch_labels = tf.reshape(batch_labels,[-1])
        task = task_labels[i]

        if task != -1:
            with tf.GradientTape() as tape:
                model = globals()[f'Task{task}_model']
                logits = model(batch_images)
                logits = tf.reshape(logits,[-1])
                loss_value = loss_fn(batch_labels, logits)
                acc = tf.keras.metrics.Accuracy()(np.round(logits),batch_labels)
                auc = tf.keras.metrics.AUC()(batch_labels,logits)
                auc = auc.numpy()
                loss_value = tf.reduce_mean(loss_value)
                loss_list[task].append(loss_value)
                acc_list[task].append(acc)
                auc_list[task].append(auc)
            grads = tape.gradient(loss_value, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        else:
            with tf.GradientTape() as tape:
                model = globals()[f'Task{task}_model']
                logits = model(batch_images)
                logits = tf.reshape(logits,[-1])
                loss_value = loss_fn_reg(logits,batch_labels)
                loss_value = tf.reduce_mean(loss_value)
                loss_list[task].append(loss_value)
                acc = tf.keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None)(logits,batch_labels)
                acc_list[task].append(acc)
            grads = tape.gradient(loss_value, model.trainable_variables)
            model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        for l in range(len(Tox_names)):
            
            temp_loss = np.average(loss_list[l][-10000:])
            temp_acc = np.average(acc_list[l][-10000:])
            temp_auc = np.average(auc_list[l][-10000:])
            if l == 0:
                text = "\rSteps : {} Task : {}, Loss: {:.4f}, acc : {:.4f}, auc : {:.4f}".format(i,l,temp_loss,temp_acc,temp_auc)
            else:
                text += "     Task : {}, Loss: {:.4f}, acc : {:.4f}, auc : {:.4f}".format(l,temp_loss,temp_acc,temp_auc)
            sys.stdout.write(text)
            sys.stdout.flush()
    # 각 에포크 종료 후 평가
    for temp_index,j in enumerate(val_task):
        j = j
        val_res = predict(globals()[f'Task{j}_model'],x_vals[temp_index],len_20s[temp_index])
        val_res = val_res.reshape(-1)
        if j != -1:
            acc = tf.keras.metrics.Accuracy()(y_vals[temp_index],np.round(val_res))
            auc_res = (AUC()(y_vals[temp_index],val_res)).numpy()
            loss = loss_fn(y_vals[temp_index],val_res)
            
            print(f'\nTask is {Tox_names[temp_index]}')
            print(f"Test accuracy: {acc}")
            print(f"Test AUC: {auc_res}")
            print(f"Test loss: {loss}\n")
        else:
            loss = loss_fn_reg(y_vals[temp_index],val_res)
            print(f'\nTask is {Tox_names[temp_index]}')
            print(f"\nTest MSE:{loss}")
    for i in range(7):
        globals()[f'Task{i}_model'].save_weights(f'./BERT/SMILE_MTL/BERT_model_task{i}')

Epoch 1/25


2024-01-03 11:29:38.791655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-01-03 11:29:39.795269: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x5630f753fc90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-03 11:29:39.795305: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-01-03 11:29:39.800980: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-03 11:29:39.944837: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Steps : 11969 Task : 0, Loss: 0.4417, acc : 0.7955, auc : 0.8660     Task : 1, Loss: 0.1973, acc : 0.9305, auc : 0.7510     Task : 2, Loss: 0.4085, acc : 0.8176, auc : 0.8451     Task : 3, Loss: 0.4782, acc : 0.7884, auc : 0.7505     Task : 4, Loss: 0.4839, acc : 0.7622, auc : 0.8411     Task : 5, Loss: 0.5215, acc : 0.7497, auc : 0.6939     Task : 6, Loss: 0.4428, acc : 0.7856, auc : 0.6214
Task is AMES
Test accuracy: 0.8255494236946106
Test AUC: 0.89365553855896
Test loss: 0.4091382324695587


Task is ClinTox
Test accuracy: 0.9324324131011963
Test AUC: 0.8278035521507263
Test loss: 0.2092052698135376


Task is hERG
Test accuracy: 0.8320610523223877
Test AUC: 0.867479681968689
Test loss: 0.4131025969982147


Task is DILI
Test accuracy: 0.75789475440979
Test AUC: 0.8145518898963928
Test loss: 0.5331652760505676


Task is hERG_Karim
Test accuracy: 0.7954630255699158
Test AUC: 0.8796931505203247
Test loss: 0.43764448165893555


Task is Skin Reaction
Test accuracy: 0.6913580298423767
Test

In [2]:
import pickle
with open('./BERT/SMILE/1M_random_ZINC_word2index.pkl','rb') as file:
    temp = pickle.load(file)

In [113]:
val_res = predict(globals()[f'Task{2}_model'],x_vals[2],len_20s[2])
val_res = val_res.reshape(-1)
loss = loss_fn(y_vals[2],val_res)
loss


<tf.Tensor: shape=(), dtype=float32, numpy=0.46617275>

In [116]:
tf.keras.metrics.Accuracy()(y_vals[2],np.round(val_res))

<tf.Tensor: shape=(), dtype=float32, numpy=0.83206105>

In [ ]:
y_vals[temp_index]

array([1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [119]:
idx2word

{1: '<start>',
 2: '<end>',
 3: '<unknown1>',
 4: '<unknown2>',
 5: '<unknown3>',
 6: '<unknow4>',
 7: '<unknown5>',
 8: 'C',
 9: 'n',
 10: '1',
 11: 'c',
 12: '(',
 13: 'N',
 14: '2',
 15: '[C@H]',
 16: '[nH]',
 17: 'S',
 18: ')',
 19: '=',
 20: 'O',
 21: '[C@@H]',
 22: 'F',
 23: '[O-]',
 24: '[N@@H+]',
 25: '3',
 26: '[N@H+]',
 27: 'o',
 28: '4',
 29: 'Br',
 30: '[nH+]',
 31: '[NH+]',
 32: '5',
 33: '[C@]',
 34: '[C@@]',
 35: 's',
 36: '-',
 37: '[NH3+]',
 38: '/',
 39: '\\',
 40: 'Cl',
 41: '[N+]',
 42: '#',
 43: '[N-]',
 44: '[NH2+]',
 45: '[P@]',
 46: '[P@@]',
 47: '[n-]',
 48: '[n+]',
 49: '[N@+]',
 50: '[N@@+]',
 51: 'I',
 52: 'P',
 53: '[PH+]',
 54: '6',
 55: '[S@@]',
 56: '[Si]',
 57: '[S@]',
 58: '[o+]',
 59: '[S-]',
 60: '[O+]',
 61: '7',
 62: '[S@@+]',
 63: '[S@+]',
 64: '[S+]',
 65: '[P@H]',
 66: '[P@@H]',
 67: '[SH]',
 68: '[s+]',
 69: '[P+]',
 70: '[PH]',
 71: '8',
 72: '[Hg]',
 73: '.',
 74: '[Na]',
 75: '[Cl+3]',
 76: '[cH-]',
 77: '[CH-]',
 78: '[CH2-]',
 79: '[C-]',


In [118]:
idx2word = {}


for i in word2idx.keys():
    idx2word[word2idx[i]] = i

In [123]:
for index,i in enumerate(val_res):
    if i>0.4 and i < 0.6:
        temp = ''
        for j in x_val[index]:
            try:
                temp+=idx2word[j]
            except:
                pass
        print(temp)

<start>[C@H]1([C@H](CC[C@H]2CN3[C@@H](C[C@H]12)c1c(c2c(cccc2)[nH]1)CC3)O)C(OC)=O
<start>C1[C@H]2[C@H](C[C@H]3c4c(c5c(cccc5)[nH]4)CCN13)[C@H](C(OC)=O)[C@@H](O)CC2
<start>c1cccc2c3CCN4C[C@H]5[C@H](C[C@H]4c3[nH]c12)[C@H](C(OC)=O)[C@H](CC5)O
<start>c12[nH]c3cc(OC)ccc3c2CCN2[C@@H]1C[C@@H]1[C@H](C(O)=O)[C@@H](OC)[C@H](O)C[C@@H]1C2
<start>[C@H]12C3CCC([C@H]1NCCC2)[C@H]1N3CCc2c3ccccc3[nH]c21
<start>c1ccc(c2ccccc21)NCCN
<start>c1cc(N2N=C(C(C2=O)/N=N/c2ccc(cc2)S(=O)(O)=O)C(O)=O)ccc1S(=O)(=O)O
<start>N(\c1ccc(cc1)S(=O)(=O)O)=N/C1C(=NN(C1=O)c1ccc(cc1)S(=O)(=O)O)C(=O)O


In [125]:
for index,i in enumerate(val_res):
    if i > 0.9:
        temp = ''
        for j in x_val[index]:
            try:
                temp+=idx2word[j]
            except:
                pass
        print(temp)

<start>C(\C=O)=C\O
<start>O/C=C\C=O
<start>C(=O)/C=C\O
<start>[C@@H]1(CC[C@H]2CN3CCc4c5c([nH]c4[C@@H]3C[C@@H]2[C@@H]1C(OC)=O)cccc5)O
<start>C1CN2C[C@H]3[C@@H]([C@@H]([C@H](CC3)O)C(=O)OC)C[C@H]2c2[nH]c3ccccc3c12
<start>[C@H]12[C@@H](CC[C@@H]([C@H]1C(OC)=O)O)CN1[C@@H](C2)c2c(c3c(cccc3)[nH]2)CC1
<start>C([C@H]1[C@H]2C[C@@H]3N(CCc4c5c([nH]c43)cccc5)C[C@@H]2CC[C@@H]1O)(=O)OC
<start>c1c2c3c([C@H]4N(CC3)C[C@@H]3CC[C@H](O)[C@H]([C@H]3C4)C(OC)=O)[nH]c2ccc1
<start>COC([C@@H]1[C@H](CC[C@@H]2[C@@H]1C[C@@H]1N(CCc3c1[nH]c1c3cccc1)C2)O)=O
<start>c12c3c([C@@H]4C[C@@H]5[C@@H]([C@@H](O)CC[C@H]5CN4CC3)C(OC)=O)[nH]c2cccc1
<start>[C@H]1(O)CC[C@H]2CN3[C@@H](C[C@@H]2[C@@H]1C(OC)=O)c1[nH]c2c(cccc2)c1CC3
<start>O[C@H]1CC[C@@H]2[C@@H]([C@@H]1C(OC)=O)C[C@H]1c3[nH]c4ccccc4c3CCN1C2
<start>[C@@H]1(O)[C@@H](C(OC)=O)[C@@H]2[C@H](CN3[C@@H](C2)c2[nH]c4ccccc4c2CC3)CC1
<start>O=C(OC)[C@@H]1[C@@H](O)CC[C@@H]2[C@@H]1C[C@@H]1N(C2)CCc2c3ccccc3[nH]c12
<start>C1[C@@H]2c3[nH]c4cc(OC)ccc4c3CCN2C[C@@H]2[C@H]1[C@H](C(=O)O)[C@H]([C

In [108]:
for j,j in enumerate(val_task):
    val_res = predict(globals()[f'Task{j}_model'],x_vals[j],len_20s[j])
    if j != -1:
        acc = tf.keras.metrics.Accuracy()(y_vals[j],np.round(val_res))
        auc_res = (AUC()(y_vals[j],val_res)).numpy()
        loss = tf.keras.metrics.BinaryCrossentropy()(y_vals[j],val_res)
        
        print(f'\nTask is {Tox_names[j]}')
        print(f"Test accuracy: {acc}")
        print(f"Test AUC: {auc_res}")
        print(f"Test loss: {loss}\n")
    else:
        loss = tf.keras.metrics.MeanSquaredError()(y_vals[j],val_res)
        print(f'\nTask is {Tox_names[j]}')
        print(f"\nTest MSE:{loss}")


Task is AMES
Test accuracy: 0.7767857313156128
Test AUC: 0.8451955318450928
Test loss: 0.48885905742645264


Task is ClinTox
Test accuracy: 0.9256756901741028
Test AUC: 0.7087756991386414
Test loss: 0.25217902660369873


Task is hERG
Test accuracy: 0.7938931584358215
Test AUC: 0.85514897108078
Test loss: 0.48679855465888977


Task is DILI
Test accuracy: 0.7157894968986511
Test AUC: 0.8307453393936157
Test loss: 0.5598265528678894


Task is hERG_Karim
Test accuracy: 0.7188546061515808
Test AUC: 0.7918149828910828
Test loss: 0.5562483668327332


Task is Skin Reaction
Test accuracy: 0.6419752836227417
Test AUC: 0.6545751690864563
Test loss: 0.616245687007904


Task is Carcinogens_Lagunin
Test accuracy: 0.8928571343421936
Test AUC: 0.9235773086547852
Test loss: 0.3122290074825287

